In [1]:
from hlda.sampler import HierarchicalLDA

In [3]:
import pandas as pd

data = pd.read_csv("matrix_data.csv",header=None)
pd_vocab = pd.read_csv("words_dict.csv",header=None)
idx = data.sum()==1
validcol = [col for i,col in enumerate(data.columns) if not idx[i] ]
validwords = [word for i,word in enumerate(list(pd_vocab[0])) if not idx[i] ]
reduced_data = data[validcol]
reduced_data.columns = range(reduced_data.shape[1])

In [14]:
len(validwords)

4469

In [18]:
# get the data vector for each document 
# for doc in corpus:
#     new_doc = []
#     for word in doc:
#         word_idx = vocab_index[word]
#         new_doc.append(word_idx)
#     new_corpus.append(new_doc)
new_corpus = []
#index is the row index, row is the document 
for index, row in reduced_data.iterrows():
    new_doc = []
    for i in range(len(validwords)):
        if row[i]!= 0:
            for j in range(row[i]):
                new_doc.append(i)
    new_corpus.append(new_doc)

In [23]:
len(new_corpus)

14640

In [26]:
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 10          # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [27]:
hlda = HierarchicalLDA(new_corpus, validwords, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling
.................................................. 50
topic 0 (level=0, total_words=56601, documents=14640): not, flight, no, us, bag, plane, day, just, make, tri, 
    topic 1 (level=1, total_words=2769, documents=983): flight, not, get, thank, can, hour, cancel, help, now, custom, 
        topic 2 (level=2, total_words=213, documents=52): reflight, problem, book, fail, line, flightl, sea, type, problemss, etc, 
        topic 11 (level=2, total_words=15, documents=7): unbeliev, day, fun, explan, promis, friend, upset, continu, certif, 3x, 
        topic 13 (level=2, total_words=123, documents=37): car, rent, rental, drive, cover, tuesday, cost, fair, thursday, erj, 
        topic 22 (level=2, total_words=16, documents=9): movi, life, tick, lima, c, overh, rerout, throughout, journey, 8b, 
        topic 32 (level=2, total_words=258, documents=38): ceo, wall, s, passeng, street, appeas, battl, republican, waterburi, right, 
        topic 40 (level=2, total_words

KeyboardInterrupt: 

In [28]:
hlda.calculate_doc_likelihood(node_weights ={}, level_word_counts = {})

KeyError: 1

In [36]:
from ipywidgets import widgets
from IPython.core.display import HTML, display
colour_map = {
    0: 'blue',
    1: 'red',
    2: 'green'
}

def show_doc(d=0):
    
    node = hlda.document_leaves[d]
    path = []
    while node is not None:
        path.append(node)
        node = node.parent
    path.reverse()   
    
    n_words = 10
    with_weights = False    
    for n in range(len(path)):
        node = path[n]
        colour = colour_map[n] 
        msg = 'Level %d Topic %d: ' % (node.level, node.node_id)
        print msg
#         msg += node.get_top_words(n_words, with_weights)
#         output = '<h%d><span style="color:%s">%s</span></h3>' % (n+1, colour, msg)
#         display(HTML(output))
        
#     display(HTML('<hr/><h5>Processed Document</h5>'))

#     doc = new_corpus[d]
#     output = ''
#     for n in range(len(doc)):
#         w = doc[n]
#         l = hlda.levels[d][n]
#         colour = colour_map[l]
#         output += '<span style="color:%s">%s</span> ' % (colour, w)
#     display(HTML(output))

In [44]:
#use the document_leaves, we can get all the level and the corresponding topic id. 
hlda.document_leaves[0].node_id

368

In [37]:
widgets.interact(show_doc, d=(0, len(new_corpus)-1))